In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('inputs/AvailabilityReport_2022-10-01.csv',header=0,dtype={'SKU': object})
df.head(2)

,Category,SKU,ProductName,Location,Bin,BatchSerialNumber,ExpiryDate,Discount,StockValue,OnHand,...,Allocated,Status,Unit,Family,StockLocator,Brand,PriceTier1,PriceTier2,PriceTier3,NextDeliveryDate
0,Non Alcoholic Beverage,777823111113,1 Water 355ml,Airport,NaN,NaN,NaN,NaN,23.4520,117.0,...,0.0,ACTIVE,Item,NaN,NaN,NaN,1.5,0.95,0.95,NaN
1,Non Alcoholic Beverage,777823111113,1 Water 355ml,Oak Hill,NaN,NaN,NaN,NaN,0.5357,1.0,...,0.0,ACTIVE,Item,NaN,NaN,NaN,1.5,0.95,0.95,NaN


In [3]:
df = df[df.Status=='ACTIVE']
#df[df.Status!='ACTIVE']

In [4]:
itnl_whiskey = ['Whiskey French', 'Whiskey Indian', 'Whiskey Mexican', 'Whiskey Taiwaneese', 'Whisky German']
df['Category'] = df.apply(lambda row: "Itnl Whiskey" if row.Category in itnl_whiskey else row.Category,axis=1)
#df[df.Category == 'Itnl Whiskey'].head(2)

,Category,SKU,ProductName,Location,Bin,BatchSerialNumber,ExpiryDate,Discount,StockValue,OnHand,...,Allocated,Status,Unit,Family,StockLocator,Brand,PriceTier1,PriceTier2,PriceTier3,NextDeliveryDate
198,Itnl Whiskey,7503027709251,Abasolo Corn Whisky 50ml,Airport,NaN,NaN,NaN,NaN,137.3583,53.0,...,0.0,ACTIVE,Item,NaN,NaN,NaN,3.99,3.09,3.09,NaN
199,Itnl Whiskey,7503027709251,Abasolo Corn Whisky 50ml,Oak Hill,NaN,NaN,NaN,NaN,23.3250,9.0,...,0.0,ACTIVE,Item,NaN,NaN,NaN,3.99,3.09,3.09,NaN


In [5]:
pt = df.pivot_table(columns=['Location'],index='ProductName',values='OnHand',aggfunc='sum').reset_index()
#pt = pt.set_index('ProductName').reset_index()
pt.head()
pt = pt.fillna(-1)
#pt.to_csv('outputs/DELETEMEWHHHELPER.csv')

In [6]:
thing = pd.merge(pt,df[['ProductName','Category']],on='ProductName', how='left').drop_duplicates(keep='first')
thing.head()

,ProductName,Airport,Oak Hill,Pleasant Valley,Warehouse,Category
0,1 Water 355ml,117.0,1.0,5.0,-1.0,Non Alcoholic Beverage
3,11 Below Brewing Taco Tuesday Cans 6 Pack,-1.0,-1.0,-1.0,0.0,Craft Beer
4,12/24 Mixing Tin Sets,0.0,-1.0,0.0,-1.0,Bar Tools
6,123 Organic Anejo Tequila 750ml,0.0,1.0,1.0,4.0,Tequila
10,123 Organic Blanco Tequila 750ml,0.0,1.0,2.0,6.0,Tequila


In [7]:
thing = thing[(thing['Pleasant Valley'] <= 0 | thing['Pleasant Valley'].isna()) & (thing['Warehouse'] > 0)]

In [8]:
thing = thing[['ProductName','Category','Pleasant Valley','Warehouse']]
thing.to_csv('outputs/ItemsToSnagFromWH.csv',index=False)